## Tratamento do Dataset

O dataset escolhido apresenta uma grande disparidade entre as categorias, então precisamos balancear isso. Para o nosso estudo de caso, vamos reduzir as categorias que possuem mais de 3000 exemplos para 3000, e realizar data augmentation nas que possuem menos exemplos.

![Quantidade de dados](imgs/qtd_dados.png)

In [ ]:
import os
import random

main_dir = 'Data'

for folder in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder)
    if os.path.isdir(folder_path):
        files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        if len(files) > 3000:
            keep_files = set(random.sample(files, 3000))
            for f in files:
                if f not in keep_files:
                    os.remove(os.path.join(folder_path, f))
            print(f"Pasta '{folder}' reduzida para 3000 arquivos.")
        else:
            print(f"Pasta '{folder}' tem {len(files)} arquivos, nada a fazer.")

Pasta 'Mild Dementia' reduzida para 3000 arquivos.
Pasta 'Very mild Dementia' reduzida para 3000 arquivos.
Pasta 'Moderate Dementia' reduzida para 3000 arquivos.
Pasta 'Non Demented' reduzida para 3000 arquivos.


## Verificando o Tamanho das Imagens

Antes de prosseguir, vamos analisar as dimensões das imagens do dataset para identificar possíveis variações de tamanho. Caso existam diferentes resoluções, será necessário realizar um redimensionamento (reshape) para padronizar todas as imagens.

In [ ]:
from PIL import Image

main_dir = 'Data'

for folder in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder)
    if os.path.isdir(folder_path):
        print(f"\nPasta: {folder}")
        tamanhos_vistos = set()
        for f in os.listdir(folder_path):
            file_path = os.path.join(folder_path, f)
            if os.path.isfile(file_path):
                try:
                    with Image.open(file_path) as img:
                        if img.size not in tamanhos_vistos:
                            print(f"{f}: {img.size}") 
                            tamanhos_vistos.add(img.size)
                except Exception as e:
                    print(f"Erro ao abrir {f}: {e}")


Pasta: Mild Dementia
aug_276_OAS1_0052_MR1_mpr-2_129.jpg: (496, 248)

Pasta: Very mild Dementia
OAS1_0023_MR1_mpr-3_127.jpg: (496, 248)

Pasta: Moderate Dementia
aug_3983_OAS1_0351_MR1_mpr-4_111.jpg: (496, 248)

Pasta: Non Demented
OAS1_0234_MR1_mpr-4_136.jpg: (496, 248)


## Começando o Pré-processamento

Como primeiro passo do pré-processamento, vamos converter todas as imagens do dataset para preto e branco. Isso garante uniformidade nos dados e facilita o processamento nas etapas seguintes.

In [ ]:
from PIL import Image

main_dir = 'Data'

for folder in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder)
    if os.path.isdir(folder_path):
        for f in os.listdir(folder_path):
            file_path = os.path.join(folder_path, f)
            if os.path.isfile(file_path):
                try:
                    with Image.open(file_path) as img:
                        img = img.convert('L')  # Converte para preto e branco
                        img.save(file_path)     # Sobrescreve a imagem original
                except Exception as e:
                    print(f"Erro ao converter {f}: {e}")

## Data Augmentation

Agora, vamos aplicar técnicas de data augmentation para gerar mais exemplos nas classes com menos imagens, ajudando a balancear o dataset e melhorar a capacidade de generalização do modelo.

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import random

main_dir = 'Data'
target_size = (248, 496)  # (altura, largura)
target_count = 3000

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
])

for folder in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder)
    if os.path.isdir(folder_path):
        files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        n_files = len(files)
        if n_files < target_count:
            print(f"Pasta '{folder}' tem {n_files} arquivos. Fazendo data augmentation...")
            i = 0
            while n_files + i < target_count:
                f = random.choice(files)
                file_path = os.path.join(folder_path, f)
                try:
                    with Image.open(file_path) as img:
                        img = img.convert('L')
                        img = img.resize(target_size[::-1])  # PIL usa (largura, altura)
                        arr = np.array(img)[..., np.newaxis]
                        arr = arr / 255.0
                        arr = np.expand_dims(arr, 0)
                        aug_img = data_augmentation(arr, training=True)
                        aug_img = tf.squeeze(aug_img).numpy()
                        aug_img = (aug_img * 255).astype(np.uint8)
                        aug_img = np.squeeze(aug_img)
                        aug_pil = Image.fromarray(aug_img, mode='L')
                        new_name = f"aug_{i}_{f}"
                        aug_pil.save(os.path.join(folder_path, new_name))
                        i += 1
                except Exception as e:
                    print(f"Erro ao abrir {f}: {e}")
            print(f"Pasta '{folder}' aumentada para {target_count} arquivos.")
        else:
            print(f"Pasta '{folder}' já tem {n_files} arquivos ou mais.")

Pasta 'Mild Dementia' já tem 3000 arquivos ou mais.
Pasta 'Very mild Dementia' já tem 3000 arquivos ou mais.
Pasta 'Moderate Dementia' já tem 3000 arquivos ou mais.
Pasta 'Non Demented' já tem 3000 arquivos ou mais.


## Verificação Final

Agora, vamos conferir se todas as pastas possuem a quantidade correta de imagens, se todas estão em preto e branco e se os tamanhos das imagens estão padronizados. Essa verificação garante que o dataset está pronto para ser utilizado nas próximas etapas do processamento.

In [ ]:
from PIL import Image

main_dir = 'Data'

for folder in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder)
    if os.path.isdir(folder_path):
        files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        tamanhos_vistos = set()
        coloridas = 0
        for f in files:
            file_path = os.path.join(folder_path, f)
            try:
                with Image.open(file_path) as img:
                    tamanhos_vistos.add(img.size)
                    if img.mode != 'L':
                        coloridas += 1
            except Exception as e:
                print(f"Erro ao abrir {f}: {e}")
        print(f"Pasta '{folder}': {len(files)} imagens")
        print(f"Tamanhos encontrados: {tamanhos_vistos}")
        if coloridas == 0:
            print("Todas as imagens estão em preto e branco.\n")
        else:
            print(f"{coloridas} imagens NÃO estão em preto e branco.\n")

Pasta 'Mild Dementia': 3000 imagens
Tamanhos encontrados: {(496, 248)}
Todas as imagens estão em preto e branco.

Pasta 'Very mild Dementia': 3000 imagens
Tamanhos encontrados: {(496, 248)}
Todas as imagens estão em preto e branco.

Pasta 'Moderate Dementia': 3000 imagens
Tamanhos encontrados: {(496, 248)}
Todas as imagens estão em preto e branco.

Pasta 'Non Demented': 3000 imagens
Tamanhos encontrados: {(496, 248)}
Todas as imagens estão em preto e branco.

